In [11]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from math import pi
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from xgboost import XGBRegressor
import holidays
from datetime import timedelta

np.set_printoptions(threshold=np.nan)

In [12]:
data =pd.read_csv('./actuals.csv')

train_cut = int(len(data) * 0.8)
validate_cut = int(len(data) * 0.9)

train, validate, test = data.iloc[0:train_cut], data.iloc[train_cut-7*24:validate_cut], data.iloc[validate_cut-7*24:]

In [13]:
X_train, y_train = train, train['rides']
X_validate, y_validate = validate, validate['rides']
X_train.head()

,date,sunrise,icon,precip_prob,temperature,humidity,wind_speed,rides
0,2013-06-01 00:00:00,0,clear,0.01,77.65,0.61,2.06,152
1,2013-06-01 01:00:00,0,clear,0.01,75.62,0.67,1.93,102
2,2013-06-01 02:00:00,0,clear,0.01,74.72,0.70,2.31,67
3,2013-06-01 03:00:00,0,clear,0.01,73.32,0.76,2.16,41
4,2013-06-01 04:00:00,0,clear,0.01,72.42,0.79,1.93,16


<h1>Dummy Predictor</h1>

It is helpful to start with a heuristic predictor. The type of predictor that can be used without any statistical or machine learning knowledge. We can compare our ML work against this predictor to determine what type of improvement machine learning has added. 

In this heuristic I am simply making predictions by going back in time 1 year and finding the closest day of the week and resuing that number. That is, I am using the 3rd Wednesday in June 2015 to predict the 3rd Wednesday in June 2016. In the event that that information is not available, I simply take the last available data from the same hour of the day.

In [4]:
class YearAgoRegressor(BaseEstimator, ClassifierMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.X = X.loc[:, ['date', 'hour']]
        self.X['date'] = self.X['date'].map(lambda x: pd.to_datetime(x))
        self.X['rides'] = y
        return self
    
    def _meaning(self, x):
        prev_year = pd.to_datetime(x.date) - pd.DateOffset(years=1)
        day_delta = int(prev_year.strftime('%w')) + 1 - x.day_of_week
        prev_year = prev_year - pd.Timedelta(days=day_delta)
        if not self.X[(self.X.date == prev_year) & (self.X.hour == x.hour)].empty:
            return self.X[(self.X.date == prev_year) & (self.X.hour == x.hour)].iloc[0, :].rides
        return self.X[(self.X.hour == x.hour)].iloc[-1, :].rides
    
    def predict(self, X, y=None):
        return(X.apply(self._meaning, axis=1))

In [5]:
params = {}
pipeline = make_pipeline(
    YearAgoRegressor()
)
clf = GridSearchCV(pipeline, params, cv=TimeSeriesSplit(3), scoring='neg_mean_squared_error', n_jobs=-1, verbose=5)
clf.fit(train, train.rides)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.4min finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None, steps=[('yearagoregressor', YearAgoRegressor())]),
       fit_params=None, iid='warn', n_jobs=-1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=5)

In [6]:
clf.score(X_validate, y_validate)

-843615.1207858355

A MSE of ~843615 will serve as my baseline

<h1>SVM</h1>

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):

    def __init__(self, columns=[]):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.loc[:, self.columns]

class CustomTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, func):
        self.func = func
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return self.func(X)

In [8]:
pipeline = Pipeline([
    ('union', FeatureUnion([
        ('cat', Pipeline([
            ('cat_selector', ColumnSelector(['sunrise'])),
            ('cat_ohe', OneHotEncoder(sparse=False))
        ])),
        ('year', Pipeline([
            ('year_selector', ColumnSelector(['date'])),
            ('year_extractor', CustomTransformer(lambda y: y.applymap(lambda x: float(pd.to_datetime(x).year - 2013)))),
            ('year_scaler', StandardScaler())
        ])),
        ('int', Pipeline([
            ('int_selector', ColumnSelector(['precip_prob', 'temperature', 'humidity', 'wind_speed', 'lag7', 'lag1'])),
            ('int_scaler', StandardScaler())
        ])),
        ('rad', Pipeline([
            ('rad_selector', ColumnSelector(['hour', 'day_of_week', 'month'])),
            ('rad_cos_sin', FeatureUnion([
                ('rad_cos', CustomTransformer(lambda y: y.apply(lambda x: np.round(np.cos(x * pi * 2/ x.nunique()), 5)))),
                ('rad_sin', CustomTransformer(lambda y: y.apply(lambda x: np.round(np.sin(x * pi * 2/ x.nunique()), 5))))
            ]))  
        ]))
    ])),
    ('svr', SVR())
])

In [9]:
params = {'svr__C': [500],
          'svr__gamma': [0.1]}
clf = GridSearchCV(pipeline, params, cv=TimeSeriesSplit(3), scoring='neg_mean_squared_error', n_jobs=-1, verbose=5)
clf.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.2min finished
C:\Users\Brian\Anaconda3\envs\citibike-dashboard\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('cat', Pipeline(memory=None,
     steps=[('cat_selector', ColumnSelector(columns=['sunrise'])), ('cat_ohe', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
   ...
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'svr__C': [500], 'svr__gamma': [0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=5)

In [10]:
clf.score(X_validate, y_validate)

-399577.7994155443

<h1>Random Forest</h1>

In [180]:
pipeline = Pipeline([
    ('union', FeatureUnion([
        ('cat', Pipeline([
            ('cat_selector', ColumnSelector(['sunrise'])),
            ('cat_ohe', OneHotEncoder(sparse=False))
        ])),
        ('year', Pipeline([
            ('year_selector', ColumnSelector(['date'])),
            ('year_extractor', CustomTransformer(lambda y: y.applymap(lambda x: float(pd.to_datetime(x).year))))
        ])),
        ('int', Pipeline([
            ('int_selector', ColumnSelector(['precip_prob', 'temperature', 'humidity', 'wind_speed', 'lag7', 'lag1']))
        ])),
        ('rad', Pipeline([
            ('rad_selector', ColumnSelector(['hour', 'day_of_week', 'month'])),
            ('rad_cos_sin', FeatureUnion([
                ('rad_cos', CustomTransformer(lambda y: y.apply(lambda x: np.round(np.cos(x * pi * 2/ x.nunique()), 5)))),
                ('rad_sin', CustomTransformer(lambda y: y.apply(lambda x: np.round(np.sin(x * pi * 2/ x.nunique()), 5))))
            ]))  
        ]))
    ])),
#     ('rfr', RandomForestRegressor())
])

In [15]:
params = {'rfr__n_estimators': [500],
          'rfr__max_depth': [20],
          'rfr__max_features': ['sqrt'],
         }
clf = GridSearchCV(pipeline, params, cv=TimeSeriesSplit(5), scoring='neg_mean_squared_error', n_jobs=-1, verbose=5)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   13.9s remaining:   20.9s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   38.2s finished
C:\Users\Brian\Anaconda3\envs\citibike-dashboard\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('cat', Pipeline(memory=None,
     steps=[('cat_selector', ColumnSelector(columns=['sunrise'])), ('cat_ohe', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
   ...s='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'rfr__n_estimators': [500], 'rfr__max_depth': [20], 'rfr__max_features': ['sqrt']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=5)

In [16]:
clf.score(X_validate, y_validate)

-369508.5522853572

In [17]:
for i in sorted(zip(['sunrise', 'sunrise', 'date', 'precip', 'temp',
             'humidity', 'wind_speed', 'lag7', 'lag1', 'hour', 'hour', 'day_of_week', 'day_of_week', 'month', 'month']
             , clf.best_estimator_.named_steps['rfr'].feature_importances_), key=lambda x: x[1], reverse=True):
    print(i)

('lag7', 0.3463949735035121)
('lag1', 0.17154210262273634)
('day_of_week', 0.06590111463786616)
('sunrise', 0.06363612025506829)
('hour', 0.062128882638282645)
('temp', 0.06184890902621661)
('sunrise', 0.0584629724357565)
('precip', 0.033230805218876835)
('humidity', 0.031512837101652216)
('date', 0.024000556872802285)
('month', 0.0191005571724678)
('hour', 0.018753165317310465)
('month', 0.017347952947530405)
('wind_speed', 0.015106824140427468)
('day_of_week', 0.011032226109493358)


<h1>XG Boost</h1>

In [14]:
class HolidaySelector(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        hd = [date for date, name in holidays.US(years=[2013, 2014, 2015, 2016, 2017, 2018]).items()
                        if name.startswith(("New Year's Day", "Washington's Birthday", "Memorial Day", "Independence Date",
                        "Labor Day", "Thanksgiving", "Christmas Day"))]
        hd_eve = [day - timedelta(days=1) for day in hd]
        hd.extend(hd_eve)
        self.h = [str(date) for date in hd]
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[['date']].applymap(lambda x: int(pd.to_datetime(x).strftime('%Y-%m-%d') in self.h))
    

class ColumnSelector(BaseEstimator, TransformerMixin):

    def __init__(self, columns=[]):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.loc[:, self.columns]
    

class DateTimeExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, extract):
        self.extract = extract
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X[['date']].applymap(lambda x: float(getattr(pd.to_datetime(x), self.extract)))


class LagTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, amount):
        self.amount=amount
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[['rides']].shift(self.amount, fill_value=0)

class CosExtractor(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        self.unique = X.nunique()
        return self
        
    def transform(self, X, y=None):
        return X.apply(lambda x: np.round(np.cos(x * pi * 2/ self.unique), 5), axis=1)

class SinExtractor(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        self.unique = X.nunique()
        return self
        
    def transform(self, X, y=None):
        return X.apply(lambda x: np.round(np.sin(x * pi * 2/ self.unique), 5), axis=1)

        
pipeline = Pipeline([
            ('union', FeatureUnion([
                ('holiday', HolidaySelector()),
                ('year_extractor', DateTimeExtractor('year')),
                ('weather_selector', ColumnSelector(['sunrise', 'precip_prob', 'temperature', 'humidity', 'wind_speed'])),
                ('lag7', LagTransformer(7*24)),
                ('lag2', LagTransformer(2*24)),
                ('hour_pipeline', Pipeline([
                    ('hour', DateTimeExtractor('hour')), 
                    ('hour_union', FeatureUnion([
                        ('hour_cos', CosExtractor()),
                        ('sin_cos', SinExtractor())
                    ]))
                ])),
                ('month_pipeline', Pipeline([
                    ('month', DateTimeExtractor('month')), 
                    ('month_union', FeatureUnion([
                        ('month_cos', CosExtractor()),
                        ('month_sin', SinExtractor())
                    ]))
                ])),
                ('dayofweek_pipeline', Pipeline([
                    ('dayofweek', DateTimeExtractor('dayofweek')), 
                    ('dayofweek_union', FeatureUnion([
                        ('dayofweek_cos', CosExtractor()),
                        ('dayofweek_sin', SinExtractor())
                    ]))
                ])),
            ])),
            ('xgbregressor', XGBRegressor())
        ])

In [15]:
params = {'xgbregressor__eta': [.005],
          'xgbregressor__max_depth': [5],
          'xgbregressor__n_estimators': [900],
          'xgbregressor__colsample_bytree': [0.5],
         }
clf = GridSearchCV(pipeline, params, cv=TimeSeriesSplit(5), scoring='neg_mean_squared_error', n_jobs=-1, verbose=5)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.3min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.9min finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('holiday', HolidaySelector()), ('year_extractor', DateTimeExtractor(extract='year')), ('weather_selector', ColumnSelector(columns=['sunrise', 'precip_prob', 'temperature', 'humidity', 'wind_speed'])), ('lag7', LagTransformer(amount...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'xgbregressor__eta': [0.005], 'xgbregressor__max_depth': [5], 'xgbregressor__n_estimators': [900], 'xgbregressor__colsample_bytree': [0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=5)

In [16]:
clf.score(X_validate, y_validate)

-265509.24243202107

In [17]:
from sklearn.externals import joblib

joblib.dump(clf.best_estimator_, 'xg.pkl', compress=1)

['xg.pkl']

In [18]:
for i in sorted(zip(['holiday', 'year', 'sunrise', 'precip', 'temperature', 'humidity', 'wind_speed', 
                     'lag7', 'lag2', 'hour_cos', 'hour_sin', 'month_cos', 'month_sin', 'dayofweek_cos', 'dayofweek_sin']
             , clf.best_estimator_.named_steps['xgbregressor'].feature_importances_), key=lambda x: x[1], reverse=True):
    print(i)

('lag7', 0.13809206)
('temperature', 0.13279441)
('lag2', 0.12423969)
('wind_speed', 0.09159047)
('humidity', 0.08362438)
('hour_cos', 0.06094259)
('hour_sin', 0.06011851)
('precip', 0.058470353)
('dayofweek_sin', 0.05839187)
('year', 0.04485343)
('month_cos', 0.04124318)
('dayofweek_cos', 0.03669113)
('month_sin', 0.035749324)
('sunrise', 0.019228505)
('holiday', 0.0139700975)
